In [1]:
#all imports
import carla

In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*firetruck*')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [3]:
# get the car's position on the map 
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-64.644844, y=24.471010, z=0.576557), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [4]:
# initial spawn point is the same - just 0.6m higher off the ground
print(start_point)

Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [5]:
#send vehicle off
# vehicle.set_autopilot(True)

In [6]:
# get actual position from the car moving
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-64.644844, y=24.471010, z=0.503709), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [7]:
# now look at the map
town_map = world.get_map()

In [8]:
type(town_map)

carla.libcarla.Map

In [9]:
print(town_map)

Map(name=Carla/Maps/Town10HD_Opt)


In [10]:
roads = town_map.get_topology()

In [11]:
# topology is pairs of waypoints defining all roads - tuples of those
print(roads)


[(<carla.libcarla.Waypoint object at 0x0000026CE55472E0>, <carla.libcarla.Waypoint object at 0x0000026CE55470B0>), (<carla.libcarla.Waypoint object at 0x0000026CE55473C0>, <carla.libcarla.Waypoint object at 0x0000026CE5547430>), (<carla.libcarla.Waypoint object at 0x0000026CE55474A0>, <carla.libcarla.Waypoint object at 0x0000026CE5547510>), (<carla.libcarla.Waypoint object at 0x0000026CE5547580>, <carla.libcarla.Waypoint object at 0x0000026CE55475F0>), (<carla.libcarla.Waypoint object at 0x0000026CE5547660>, <carla.libcarla.Waypoint object at 0x0000026CE55476D0>), (<carla.libcarla.Waypoint object at 0x0000026CE5547740>, <carla.libcarla.Waypoint object at 0x0000026CE55477B0>), (<carla.libcarla.Waypoint object at 0x0000026CE5547820>, <carla.libcarla.Waypoint object at 0x0000026CE5547890>), (<carla.libcarla.Waypoint object at 0x0000026CE5547900>, <carla.libcarla.Waypoint object at 0x0000026CE5547970>), (<carla.libcarla.Waypoint object at 0x0000026CE55479E0>, <carla.libcarla.Waypoint objec

In [12]:
#have a look at a waypoint - it is transform wrapped as a waypoint
print(roads[0][0])


Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))


In [13]:
len(roads)

200

In [57]:
import sys
sys.path.append('..\carla')
import heapq
import math

class AStarPlanner:
    def __init__(self, town_map, sampling_resolution):
        self.town_map = town_map
        self.sampling_resolution = sampling_resolution
        self.waypoints = town_map.generate_waypoints(sampling_resolution)
        print("intial self.waypoints:",self.waypoints[0])
    
    def heuristic(self, loc1, loc2):
        # Euclidean distance
        return math.sqrt((loc1.x - loc2.x) ** 2 + (loc1.y - loc2.y) ** 2 + (loc1.z - loc2.z) ** 2)
    
    def get_neighbors(self, waypoint,g_score):
        print("get_neigh waypoint",waypoint)
        x=g_score.get(self.extract_xyz(waypoint),-1) 
        print(x)
        return waypoint.next(self.sampling_resolution)
    def extract_xyz(self, waypoint):
        return (waypoint.transform.location.x, waypoint.transform.location.y, waypoint.transform.location.z)
    def trace_route(self, start_location, end_location):
        start_waypoint = self.town_map.get_waypoint(start_location)
        end_waypoint = self.town_map.get_waypoint(end_location)
        
        open_set = []
        heapq.heappush(open_set, (0, start_waypoint))
        
        came_from = {}
        g_score={}
        print("intial self.waypoints[1]:",self.waypoints[1])

        g_score = {self.extract_xyz(wp): (wp,float('inf')) for wp in self.waypoints}
        print("g_score",g_score)
        g_score[self.extract_xyz(start_waypoint)] = (start_waypoint,0)
        print("g_score",g_score)
        f_score = {self.extract_xyz(wp): float('inf') for wp in self.waypoints}
        # print("f_score",f_score)
        f_score[start_waypoint] = self.heuristic(start_waypoint.transform.location, end_waypoint.transform.location)
        
        while open_set:
            _, current = heapq.heappop(open_set)
            print("current",current)
            print("open_set",open_set)
            
            if current == end_waypoint:
                total_path = [current]
                while current in came_from:
                    current = came_from[current]
                    total_path.append(current)
                total_path.reverse()
                return total_path
            
            for neighbor in self.get_neighbors(current,g_score):
                # print(start_waypoint) # start and current are same at beginning
                print("1st neighbor",neighbor)
                # print("g_score",g_score[self.extract_xyz(start_waypoint)]) # start exists in g_score
                tentative_g_score = g_score[self.extract_xyz(current)] + self.heuristic(current.transform.location, neighbor.transform.location)
                if tentative_g_score < g_score.get(self.extract_xyz(neighbor),float('inf')):
                    n=self.extract_xyz(neighbor)
                    came_from[n] = self.extract_xyz(current)
                    g_score[n] = tentative_g_score
                    f_score[n] = tentative_g_score + self.heuristic(neighbor.transform.location, end_waypoint.transform.location)
                    print('updated open_set')
                    heapq.heappush(open_set, (f_score[n], n))
                print('end loop')
        return []
    


# Usage example:
sampling_resolution = 2
town_map = world.get_map()
astar_planner = AStarPlanner(town_map, sampling_resolution)

point_a = carla.Location(x=-64.644844, y=24.471010, z=0.600000)
point_b = carla.Location(x=50.477512, y=141.135620, z=0.001844)

route = astar_planner.trace_route(point_a, point_b)
for waypoint in route:
    world.debug.draw_string(waypoint.transform.location, '^', draw_shadow=False, color=carla.Color(r=0, g=255, b=0), life_time=120.0, persistent_lines=True)


intial self.waypoints: Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))
intial self.waypoints[1]: Waypoint(Transform(Location(x=106.429962, y=-9.358066, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))
g_score {(109.92987823486328, -9.334196090698242, 0.0): (<carla.libcarla.Waypoint object at 0x0000026CE6C256D0>, inf), (106.42996215820312, -9.358065605163574, 0.0): (<carla.libcarla.Waypoint object at 0x0000026CE6C25660>, inf), (102.93003845214844, -9.38193416595459, 0.0): (<carla.libcarla.Waypoint object at 0x0000026CE6C255F0>, inf), (99.43012237548828, -9.405803680419922, 0.0): (<carla.libcarla.Waypoint object at 0x0000026CE6C25580>, inf), (109.94214630126953, -11.346802711486816, 0.0): (<carla.libcarla.Waypoint object at 0x0000026CE6C25510>, inf), (106.44218444824219, -11.362241744995117, 0.0): (<carla.libcarla.Waypoint object at 0x0000026CE6C254A0>, inf), (102.94221496582031, -11.37768

AttributeError: 'tuple' object has no attribute 'transform'

In [48]:
# making a route
#from one position to another
#pos 1: Transform(Location(x=50.477512, y=141.135620, z=0.001844), Rotation(pitch=0.000007, yaw=0.318098, roll=0.000000))
#pos 2: Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))

In [159]:
# utility script of destruction

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [ ]:
# Ignore below this cell ###########################################################################################

In [50]:
# import some code coming with the sim
import sys
sys.path.append('..\carla')
# sys.path.append('D:\WindoesNoEditor\PythonAPI\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [51]:
# using the code to plan the route and then draw it in the simulator
#town10hd_map = world.get_map()

sampling_resolution = 2
grp= GlobalRoutePlanner(town_map, sampling_resolution)
point_a=carla.Location(x=-64.644844, y=24.471010, z=0.600000)
point_b=carla.Location(x=50.477512, y=141.135620, z=0.001844)

route=grp.trace_route(point_a,point_b)
for waypoint in route:
    # print(waypoint)
    world.debug.draw_string(waypoint[0].transform.location,'^',draw_shadow=False, color=carla.Color(r=0, g=255, b=0), life_time=120.0, persistent_lines=True)

: 

In [18]:
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

#start first car in alredy defined start point
truck = world.try_spawn_actor(truck_bp[0], start_point)

In [19]:
# tweak spectator position to watch the show

spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4),
                                carla.Rotation(yaw = start_point.rotation.yaw -155))

spectator.set_transform(spectator_pos)

In [20]:
# drop the Mini the sky - watch what happens after

#spawn it first somewhere else
mini = world.try_spawn_actor(mini_bp[0], spawn_points[10])

mini_pos = carla.Transform(start_point.location + carla.Location(x=-4,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
mini.set_transform(mini_pos)


In [21]:
import time
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

#start first car in alredy defined start point
truck = world.try_spawn_actor(truck_bp[0], start_point)
time.sleep(3)

vw_bp = world.get_blueprint_library().filter('*volkswagen*')

vw = world.try_spawn_actor(vw_bp[0], spawn_points[20])
vw_pos = carla.Transform(start_point.location + carla.Location(x=1,z=6),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
vw.set_transform(vw_pos)
mini = world.try_spawn_actor(mini_bp[0], spawn_points[10])

mini_pos = carla.Transform(start_point.location + carla.Location(x=1,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
mini.set_transform(mini_pos)


In [22]:
vw_bp = world.get_blueprint_library().filter('*volkswagen*')

In [23]:
vehicle_all = world.get_blueprint_library()
print(vehicle_all)

[ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra]), ActorBlueprint(id=vehicle.audi.a2,tags=[vehicle, audi, a2]), ActorBlueprint(id=static.prop.plantpot04,tags=[static, prop, plantpot04]), ActorBlueprint(id=static.prop.bench02,tags=[bench02, static, prop]), ActorBlueprint(id=vehicle.mercedes.coupe_2020,tags=[vehicle, mercedes, coupe_2020]), ActorBlueprint(id=vehicle.audi.tt,tags=[vehicle, audi, tt]), ActorBlueprint(id=walker.pedestrian.0037,tags=[0037, walker, pedestrian]), ActorBlueprint(id=vehicle.harley-davidson.low_rider,tags=[harley-davidson, vehicle, low_rider]), ActorBlueprint(id=vehicle.bmw.grandtourer,tags=[vehicle, bmw, grandtourer]), ActorBlueprint(id=static.prop.trashcan05,tags=[static, prop, trashcan05]), ActorBlueprint(id=walker.pedestrian.0038,tags=[walker, pedestrian, 0038]), ActorBlueprint(id=vehicle.carlamotors.firetruck,tags=[vehicle, carlamotors, firetruck]), ActorBlueprint(id=vehicle.micro.microlino,tags=[vehicle, microlino, micro]), ActorBluepri

: 